<a href="https://colab.research.google.com/github/rodmorx/DSP_DT_WS24/blob/main/DSProjekt_DT_WS24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Sience Projekt WS24 - Dream Team

In [1]:
!python -m spacy download de_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')


In [76]:
import pandas as pd
import spacy
import string
from spacy import displacy

In [135]:
df = pd.read_csv("articles_24.csv")

## Data Cleaning

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47784 entries, 0 to 47783
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         47784 non-null  object
 1   authors       47784 non-null  object
 2   date_publish  47784 non-null  object
 3   maintext      47368 non-null  object
 4   url           47784 non-null  object
 5   newspaper     47784 non-null  object
dtypes: object(6)
memory usage: 2.2+ MB


In [137]:
df.head()

,title,authors,date_publish,maintext,url,newspaper
0,Yvonne Woelke wollte ein Baby: Kehrtwende! „Ki...,['Tanja May'],2024-01-01 22:00:02,"„Ich wünsche mir nichts sehnlicher, als endlic...",https://www.bild.de/bild-plus/unterhaltung/leu...,bild.de
1,RB Leipzig: Es wurde über Instagram öffentlich...,['Yvonne Gabriel'],2024-01-01 22:36:21,"Stammplatz weg, ungewisse Zukunft. Sportlich l...",https://www.bild.de/sport/fussball/rb-leipzig/...,bild.de
2,Apple Watch 8 mit Alu-Sportband beim BILDplus ...,['BILD'],2024-01-02 08:06:51,Frohes Neues liebe BILDplus-User! Damit der St...,https://www.bild.de/gewinnspiele/bildplus-akti...,bild.de
3,Werder Bremen: „1000-mal totgesagt“ - Ein Prof...,['Markus Balczuweit'],2024-01-01 22:12:40,Christian Groß (34) ist unverwüstlich.\nWerder...,https://www.bild.de/sport/fussball/werder-brem...,bild.de
4,"Angelique Kerber: Kaum zu glauben, wer in der ...",['SEBASTIAN KAYSER'],2024-01-01 22:43:31,Tennis-Start ins Neue Jahr! In Australien spie...,https://www.bild.de/sport/mehr-sport/tennis/an...,bild.de


In [138]:
# Retrieve the 'maintext' article from the first row
article = df.loc[1, "maintext"]

# Display the article
print(article)

Stammplatz weg, ungewisse Zukunft. Sportlich läuft es für Leipzig-Star Timo Werner (27) derzeit gar nicht rund. Dafür privat aber umso besser!
Wie Freundin Paula Lense zum Jahreswechsel auf Instagram öffentlich machte, haben sie und der Stürmer sich im Sommer verlobt. Die gebürtige Dresdnerin schreibt dazu: „Dieses Jahr war so besonders. Vor sechs Monaten haben wir für immer Ja zueinander gesagt und ich könnte nicht glücklicher sein, mein ganzes Leben mit dir verbringen zu dürfen. Ja zu meinem Seelenverwandten, bestem Freund und meinem ganzen Leben. Dich mit 16 Jahren zu treffen, war wirklich das Beste, was mir je passiert ist! Sieben Jahre später sind wir bereit für das nächste Kapitel. Liebe dich für immer und immer. Bis dass der Tod uns scheidet.“
Unter dem Post gratulieren unter anderem die Spielerfrauen Sara Gündogan, Anika Neuer und Sophia Weber, Freundin von Kai Havertz.
Die Verlobung fand auf Mallorca statt, wo Werner ein Haus besitzt und im Sommer immer Urlaub macht. Paula prä

In [139]:
# Function to check if maintext is valid
def is_valid_article(text):
    # Ensure the text is a string
    if not isinstance(text, str):
        return False
    
    # Check word count
    word_count = len(text.split())
    if word_count < 300:
        return False
    
    # Check for repeated sentences
    sentences = text.split('. ')
    for i in range(len(sentences) - 1):
        if sentences[i].strip() == sentences[i + 1].strip():
            return False
    
    return True

# Apply the function to filter the DataFrame
df = df[df["maintext"].apply(is_valid_article)]

# Reset the index
df = df.reset_index(drop=True)


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11242 entries, 0 to 11241
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         11242 non-null  object
 1   authors       11242 non-null  object
 2   date_publish  11242 non-null  object
 3   maintext      11242 non-null  object
 4   url           11242 non-null  object
 5   newspaper     11242 non-null  object
dtypes: object(6)
memory usage: 527.1+ KB


In [141]:
df.head(15)

,title,authors,date_publish,maintext,url,newspaper
0,RB Leipzig: Es wurde über Instagram öffentlich...,['Yvonne Gabriel'],2024-01-01 22:36:21,"Stammplatz weg, ungewisse Zukunft. Sportlich l...",https://www.bild.de/sport/fussball/rb-leipzig/...,bild.de
1,Apple Watch 8 mit Alu-Sportband beim BILDplus ...,['BILD'],2024-01-02 08:06:51,Frohes Neues liebe BILDplus-User! Damit der St...,https://www.bild.de/gewinnspiele/bildplus-akti...,bild.de
2,"Angelique Kerber: Kaum zu glauben, wer in der ...",['SEBASTIAN KAYSER'],2024-01-01 22:43:31,Tennis-Start ins Neue Jahr! In Australien spie...,https://www.bild.de/sport/mehr-sport/tennis/an...,bild.de
3,FC Bayern: Ungewohntes Programm! So starten di...,['Nico Linner'],2024-01-01 22:15:11,Dieser Start in 2024 hat es in sich!\nHeute ha...,https://www.bild.de/sport/fussball/bayern-muen...,bild.de
4,Putin gefährlich wie nie: Brauchen wir jetzt s...,['Sebastian Geisler'],2024-01-01 21:53:49,Muss Europa JETZT drastisch aufrüsten?\nKreml-...,https://www.bild.de/politik/ausland/politik-au...,bild.de
5,CDU-Generalsekretär Carsten Linnemann fordert:...,['BILD'],2024-01-01 21:50:22,Klare Worte von Carsten Linnemann (46)!\nDer C...,https://www.bild.de/politik/inland/politik-inl...,bild.de
6,Darts-WM: Wunderkind Luke Littler macht seine ...,['Max Schrader und Jonas Ortmann'],2024-01-01 21:33:47,Luke Littler (16) steht sensationell im Halbfi...,https://www.bild.de/sport/mehr-sport/darts/dar...,bild.de
7,Praktikum bei Preußen: Daube macht den Sportfa...,['Von JOACHIM SCHUTH'],2024-01-01 20:29:11,Vor genau 233 Tagen hatte Dennis Daube (34) se...,https://www.bild.de/sport/fussball/fussball/pr...,bild.de
8,Gunnar Schupelius - Mein Ärger: Die Regierung ...,['Gunnar Schupelius'],2024-01-01 20:27:57,BERLIN – Die Klimapolitik ist einem Automatism...,https://www.bild.de/regional/berlin/berlin-akt...,bild.de
9,Darts-WM: Wunderkind Luke Littler steht im Hal...,['Max Schrader und Jonas Ortmann'],2024-01-01 20:21:05,Er macht die Sport-Welt sprachlos.\nDer 16-jäh...,https://www.bild.de/sport/mehr-sport/darts/dar...,bild.de


In [142]:
# Retrieve the 'maintext' article from the first row
article = df.loc[14, "maintext"]

# Display the article
print(article)

Welche geheime Botschaft steckt hinter diesem Post?
Zum Start ins neue Jahr wünscht Fußball-Star Toni Kroos (33) seinen Fans in den sozialen Medien „ein frohes neues Jahr“ – und lässt mit einem interessanten Satz Raum für Spekulationen.
Auf der Plattform X (ehemals Twitter) schreibt der Real-Kicker auch Folgendes: „24 wird in vielerlei Hinsicht besonders.“
Meint Kroos damit etwa auch die Europameisterschaft in Deutschland und ein damit verbundenes Comeback in unserer Nationalmannschaft? Möglich! Viele Fans würde es sicher freuen.
BILD hatte bereits Mitte Dezember davon berichtet, dass sich Kroos ein Comeback in der Nationalmannschaft vorstellen könnte. Sein Rücktritt nach dem Aus im EM-Achtelfinale gegen England 2021 („Weil ich für die nächsten Jahre die volle Konzentration auf meine Ziele mit Real Madrid richten möchte“) ist nicht mehr in Stein gemeißelt.
Zuvor hatte Bundestrainer Julian Nagelsmann (36) eine Kroos-Rückkehr bei einem Auftritt im ZDF-Sportstudio als „interessanten Gedan

In [ ]:
# Function to format authors
def format_authors(authors):
    if not isinstance(authors, str):
        return authors  # Skip non-string values
    print(authors)

    # Convert to title case
    formatted = authors.title()
    
    # Replace "und" with ", "
    formatted = formatted.replace(", ", "', '") # fix the commas and quotes position
    formatted = formatted.replace(" Und", "', '") # fix various ways to say 'und'
    formatted = formatted.replace(" &", "', '") # fix various ways to say 'und'
    formatted = formatted.replace("''", "'") # fix the double quotes
    
    # Remove "Von" or "von" from the start of words
    if formatted.startswith("['Von"):
        formatted = "['" + formatted[5:].strip() 
    
    return formatted

# Apply the function to the 'authors' column
df["authors"] = df["authors"].apply(format_authors)

In [145]:
df.head(15)

,title,authors,date_publish,maintext,url,newspaper
0,RB Leipzig: Es wurde über Instagram öffentlich...,['Yvonne Gabriel'],2024-01-01 22:36:21,"Stammplatz weg, ungewisse Zukunft. Sportlich l...",https://www.bild.de/sport/fussball/rb-leipzig/...,bild.de
1,Apple Watch 8 mit Alu-Sportband beim BILDplus ...,['Bild'],2024-01-02 08:06:51,Frohes Neues liebe BILDplus-User! Damit der St...,https://www.bild.de/gewinnspiele/bildplus-akti...,bild.de
2,"Angelique Kerber: Kaum zu glauben, wer in der ...",['Sebastian Kayser'],2024-01-01 22:43:31,Tennis-Start ins Neue Jahr! In Australien spie...,https://www.bild.de/sport/mehr-sport/tennis/an...,bild.de
3,FC Bayern: Ungewohntes Programm! So starten di...,['Nico Linner'],2024-01-01 22:15:11,Dieser Start in 2024 hat es in sich!\nHeute ha...,https://www.bild.de/sport/fussball/bayern-muen...,bild.de
4,Putin gefährlich wie nie: Brauchen wir jetzt s...,['Sebastian Geisler'],2024-01-01 21:53:49,Muss Europa JETZT drastisch aufrüsten?\nKreml-...,https://www.bild.de/politik/ausland/politik-au...,bild.de
5,CDU-Generalsekretär Carsten Linnemann fordert:...,['Bild'],2024-01-01 21:50:22,Klare Worte von Carsten Linnemann (46)!\nDer C...,https://www.bild.de/politik/inland/politik-inl...,bild.de
6,Darts-WM: Wunderkind Luke Littler macht seine ...,"['Max Schrader', ' Jonas Ortmann']",2024-01-01 21:33:47,Luke Littler (16) steht sensationell im Halbfi...,https://www.bild.de/sport/mehr-sport/darts/dar...,bild.de
7,Praktikum bei Preußen: Daube macht den Sportfa...,['Joachim Schuth'],2024-01-01 20:29:11,Vor genau 233 Tagen hatte Dennis Daube (34) se...,https://www.bild.de/sport/fussball/fussball/pr...,bild.de
8,Gunnar Schupelius - Mein Ärger: Die Regierung ...,['Gunnar Schupelius'],2024-01-01 20:27:57,BERLIN – Die Klimapolitik ist einem Automatism...,https://www.bild.de/regional/berlin/berlin-akt...,bild.de
9,Darts-WM: Wunderkind Luke Littler steht im Hal...,"['Max Schrader', ' Jonas Ortmann']",2024-01-01 20:21:05,Er macht die Sport-Welt sprachlos.\nDer 16-jäh...,https://www.bild.de/sport/mehr-sport/darts/dar...,bild.de


## Saving the dataframe to a csv file

In [144]:
df.to_csv('out.csv', index=False)